In [1]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [2]:
import sys
sys.path.append('~/aiffel/Aiffelthon_koBART')

In [3]:
!ls

'=1.10.0'				        modeling_outputs.py
 configuration_utils.py			        __pycache__
 data					        requirements.txt
'EDA result.ipynb'			        roberta-ko-small.ipynb
 fine_tune_bart_summarization_two_langs.ipynb   robertatest.ipynb
'hubbingface finetuning seq2seq.ipynb'	        seq2seq_trainer.py
 import_utils.py			        seq2seq_training_args.py
'채팅 요약기 미세 조정.ipynb'		        Untitled1.ipynb
 kobART_modelling.ipynb			        Untitled2.ipynb
 koBART_modelling.ipynb			        Untitled.ipynb
 modeling_encoder_decoder.py


In [4]:
!pip install rouge_score
!pip install datasets==1.0.2
# !pip install transformers

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=a2873342779dc3b411a55a87dc168f02bbc7f3fe9b6bc9220e20e65f4d9c8162
  Stored in directory: /aiffel/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score
     |████████████████████████████████| 1.8 MB 5.8 MB/s            
  Attempting uninstall: datasets
    Found existing installation: datasets 1.14.0
    Uninstalling datasets-1.14.0:
      Successfully uninstalled datasets-1.14.0


In [5]:
pip install transformers==4.24.0

     |████████████████████████████████| 5.5 MB 6.6 MB/s            
     |████████████████████████████████| 163 kB 84.9 MB/s            
     |████████████████████████████████| 7.6 MB 64.3 MB/s            
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
  Attempting uninstall: transformers
    Found existing installation: transformers 4.11.3
    Uninstalling transformers-4.11.3:
      Successfully uninstalled transformers-4.11.3
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install transformer-utils

     |████████████████████████████████| 1.7 MB 9.8 MB/s            
     |████████████████████████████████| 86 kB 10.7 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install packaging

Note: you may need to restart the kernel to use updated packages.


In [8]:
# 필요한 라이브러리 불러오기
import datasets
import transformers
import pandas as pd
from datasets import Dataset

#Tokenizer
from transformers import RobertaTokenizerFast

#Encoder-Decoder Model
from transformers import EncoderDecoderModel

#Training
from seq2seq_trainer import Seq2SeqTrainer
from transformers import TrainingArguments
from seq2seq_training_args import Seq2SeqTrainingArguments
from dataclasses import dataclass, field
from typing import Optional

In [9]:
# 경로 지정
#%cd ~/aiffel/aiffelthon

In [10]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/legacy/seq2seq/seq2seq_trainer.py

In [11]:
#!wget https://github.com/huggingface/transformers/blob/main/src/transformers/utils/import_utils.py

In [12]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/main/src/transformers/models/encoder_decoder/modeling_encoder_decoder.py

In [13]:
#!wget https://raw.githubusercontent.com/huggingface/transformers/main/src/transformers/configuration_utils.py

In [14]:
# 데이터 불러오기
train_df = pd.read_csv('data/train_20per.csv')
train_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)
train_df.rename(columns = {"input_documant": "Text"}, inplace = True)
train_df.rename(columns = {"sentence_20%": "Summary"}, inplace = True)

val_df = pd.read_csv('data/val_20per.csv')
val_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)
val_df.rename(columns = {"input_documant": "Text"}, inplace = True)
val_df.rename(columns = {"sentence_20%": "Summary"}, inplace = True)
print(len(train_df), len(val_df))
train_df.head()

73431 9150


,Text,Summary
0,"지어내버린 대목부터는 흥분이 버썩 줄어지었다 ──. ""선생님! 또 기침이 나고 토...",자신을 배반한 제 계집과 세상이 엎드려 죄 사하기를 빌 때까지 죽지 아니하겠다는 H...
1,이 송아지가 젖을 떼우고 집으로 끌고 오던 날은 첨지는 개선장군이 성안 에 들어올...,젖뗀 송아지를 집으로 끌고 오던 날 어깨춤을 추면서 소 들어간다고 고함을 지르는 첨...
2,어떤 여름날 밤 손주딸에게 관한 불길한 꿈을 꾼 이 한머니는 이튿날 조반 후에 생...,손주딸의 관한 불길한 꿈을 꾼 한머니는 백여 리 떨어진 손주딸의 집에를 가보기로 하...
3,이러한 가운데서 왕후는 자기의 입장을 위태롭게 여기고 겸하여 장래 자기 의 몸으로...,자기는 태자의 위를 동경하거나 부러워한 적이 없으며 이 나라의 충성된 신자로서 공주...
4,“가겠소.” “언니가 나오시면 일러드 리겠으니 그때까지는 찾아오지 않으시는 것이 ...,자신의 신변을 염려하여 빠른 걸음으로 골목을 빠져나와 침착한 의식을 회복하면서 어수...


In [15]:
# 10번째 row만 추출
train_df = train_df.iloc[range(0, 73431, 5)]
val_df = val_df.iloc[range(0, 9150, 3)]
print(len(train_df), len(val_df))

14687 3050


In [16]:
train_df.head()

,Text,Summary
0,"지어내버린 대목부터는 흥분이 버썩 줄어지었다 ──. ""선생님! 또 기침이 나고 토...",자신을 배반한 제 계집과 세상이 엎드려 죄 사하기를 빌 때까지 죽지 아니하겠다는 H...
5,"""그래라."" 용선은 선뜻 허락하는 말을 준다. ""네? 소승의 소원을 이루어주십니까...",조신이 목욕하고 새 옷을 갈아입고 관음전으로 들어가는 것을 본 용선 법사는 문을 밖...
10,“왜.” 맹서방은 수방의 눈치를 살피며 한 걸음 다가섰다. 그리고 주인 마누라에 ...,부뚜막에서 밥이 끓어날 때에야 어머니가 나왔고 수방이는 어머니가 무엇을 또 잘못했다...
15,피아노 궐(厥)은 가정의 단란(團欒)에 흠씬 심신(心身)을 잠그게 되었다. 보기만 ...,궐은 중등교육을 마쳤으며 생글생글 웃는 눈매와 날씬날씬한 허리를 가진 새 안해와의 ...
20,"“오누, 오누.” “양……” “이리 온.” 이리하여 커다란 손으로 까맹이를 움켜쥔...",춘심이는 생전에 순 서방의 기쁨을 곱 되게 하고 근심은 사라지게 하였으며 모든 것의...


In [17]:
def preprocess_sentence(sentence):
    sentence = sentence.str.lower() # 텍스트 소문자화
    sentence = sentence.str.replace(pat= r'([\n])', repl=r'', regex=True)
    sentence = sentence.str.replace(pat= r'(['""'_])', repl=r'', regex=True)
    sentence = sentence.str.replace(pat= r'([?.!])', repl=r' \1', regex=True) #  구두점 분리
    sentence = sentence.str.replace(pat=r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9!.?]+', repl =" ",regex=True) # 0-9ㄱ-ㅎㅏ-ㅣ가-힣a-z!.?가 아닌 모든 문자를 하나의 공백으로 바꿉니다
    return sentence

In [18]:
cols = ['Text','Summary']

for col in cols:
    train_df[col] = preprocess_sentence(train_df[col])
    val_df[col] = preprocess_sentence(val_df[col])

In [19]:
train_df.head()

,Text,Summary
0,지어내버린 대목부터는 흥분이 버썩 줄어지었다 . 선생님 ! 또 기침이 나고 토혈이...,자신을 배반한 제 계집과 세상이 엎드려 죄 사하기를 빌 때까지 죽지 아니하겠다는 의...
5,그래라 . 용선은 선뜻 허락하는 말을 준다 . 네 ? 소승의 소원을 이루어주십니까...,조신이 목욕하고 새 옷을 갈아입고 관음전으로 들어가는 것을 본 용선 법사는 문을 밖...
10,왜 . 맹서방은 수방의 눈치를 살피며 한 걸음 다가섰다 . 그리고 주인 마누라에 ...,부뚜막에서 밥이 끓어날 때에야 어머니가 나왔고 수방이는 어머니가 무엇을 또 잘못했다...
15,피아노 궐 은 가정의 단란 에 흠씬 심신 을 잠그게 되었다 . 보기만 하여도 지긋지...,궐은 중등교육을 마쳤으며 생글생글 웃는 눈매와 날씬날씬한 허리를 가진 새 안해와의 ...
20,오누 오누 . 양 이리 온 . 이리하여 커다란 손으로 까맹이를 움켜쥔 다음에는 논...,춘심이는 생전에 순 서방의 기쁨을 곱 되게 하고 근심은 사라지게 하였으며 모든 것의...


In [20]:
val_df.head()

,Text,Summary
0,러시아 황실이 제공한 임시공관에서 오전 내내 쉬며 지냈다 . 오늘 아침 우리 임시...,러시아에서 제공한 임시공관에서 쉬면서 아침에 태극기를 발코니 위에 게양했는데 모스크...
3,그런 점에서 볼 때 박영민씨 경우는 이례적인 케이스로 꼽힌다 . 프로게이머 특성상...,박 씨는 기초 지식이 부족해 영어 문답을 통째로 외우다시피 하며 공무원 시험 준비를...
6,지난 9월 30일 스위스 세계경제포럼 은 140개국을 상대로 한 금융시장 성숙도 조...,스위스 세계경제포럼은 140개국을 상대로 한 금융시장 성숙도 조사에서 한국이 우간다...
9,이처럼 자체 기금운용계획 변경범위를 축소한 것은 기금변경에 대한 국회의 심의대상을 ...,기획예산처는 기금운용계획 수립 당시에는 예측할 수 없었으나 시급하게 추진할 필요가 ...
12,보이어 기장은 물체들의 실체를 파악하기 위해 더 가까이 가고 싶었지만 승객들을 위...,보이어 기장은 목적지에 착륙한 뒤 승객들에게 무언가 특별한 것을 보지 못했는지 물었...


In [21]:
# reset_index 사용
train_df.reset_index(inplace=True, drop=True)
val_df.reset_index(inplace=True, drop=True)

In [22]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_df) 
val_len = len(val_df) // 2
val_data = Dataset.from_pandas(val_df[:val_len])
test_data=Dataset.from_pandas(val_df[val_len:])

In [23]:
print(train_data)
print(val_data)
print(test_data)

Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 14687)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 1525)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 1525)


In [24]:
max_input = 512
max_target = 128
batch_size = 3
model_checkpoints = "gogamza/kobart-base-v1"

In [25]:

import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk
import numpy as np
import nltk

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Downloading:   0%|          | 0.00/682k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/111 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [27]:
def preprocess_data(data_to_process):
  #get all the dialogues
  inputs = [dialogue for dialogue in data_to_process['Text']]
  #tokenize the dialogues
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)
  #tokenize the summaries
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['Summary'], max_length=max_target, padding='max_length', truncation=True)
    
  #set labels
  model_inputs['labels'] = targets['input_ids']
  #return the tokenized data
  #input_ids, attention_mask and labels
  return model_inputs

In [28]:
ttokenize_data = train_data.map(preprocess_data, batched = True)
vtokenize_data = val_data.map(preprocess_data, batched = True)

/opt/conda/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [29]:
print(ttokenize_data)
print()
print(type(ttokenize_data))

Dataset(features: {'Summary': Value(dtype='string', id=None), 'Text': Value(dtype='string', id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 14687)

<class 'datasets.arrow_dataset.Dataset'>


In [30]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [31]:
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),}

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

In [32]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=1,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,  # demo
    per_device_eval_batch_size=4,
    # learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=50,
    save_total_limit=3,
)

In [33]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

In [34]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [35]:
trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=ttokenize_data,
    eval_dataset=vtokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [36]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids, Text, Summary. If token_type_ids, Text, Summary are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14687
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 3672
  Number of trainable parameters = 123859968
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, usi

Step,Training Loss
50,14.515900
100,5.984300
150,3.753300
200,2.306300
250,1.977400
300,1.882200
350,1.829500
400,1.834400
450,1.804500
500,1.801800


Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-500/tokenizer_config.json
Special tokens file saved in results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000/config.json
Model weights saved in results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to results/checkpoint-1500
Configuration saved in results/checkpoint-1500/config.json
Model weights saved in results/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1500/tokenizer_config.json
Special tokens file saved in results/checkpoint-1500/special_tokens_map.json
Saving model check

TrainOutput(global_step=3672, training_loss=2.000308951001801, metrics={'train_runtime': 1281.8624, 'train_samples_per_second': 11.458, 'train_steps_per_second': 2.865, 'total_flos': 4477599681085440.0, 'train_loss': 2.000308951001801, 'epoch': 1.0})

In [39]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids, Text, Summary. If token_type_ids, Text, Summary are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1525
  Batch size = 4


{'eval_loss': 1.6716617345809937,
 'eval_rouge2_precision': 0.0163,
 'eval_rouge2_recall': 0.0137,
 'eval_rouge2_fmeasure': 0.0144,
 'eval_runtime': 118.1053,
 'eval_samples_per_second': 12.912,
 'eval_steps_per_second': 3.234,
 'epoch': 1.0}

In [38]:
val_data['pred']

ValueError: Column (pred) not in table columns (['Text', 'Summary']).

In [42]:
encoder_max_length = 256  # demo
decoder_max_length = 64

In [45]:
type(val_data.select(range(16)))

datasets.arrow_dataset.Dataset

In [46]:
test_samples = val_data.select(range(16))
test_samples

Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 16)

In [48]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["Text"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    print('통과2')
    input_ids = inputs.input_ids.to(model.device)
    
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained("gogamza/kobart-base-v1")

test_samples = val_data.select(range(16))

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model_before_tuning)[1]

loading configuration file config.json from cache at /aiffel/.cache/huggingface/hub/models--gogamza--kobart-base-v1/snapshots/d7e64abd841bc1fa5d2939d14161124c51f29e8b/config.json
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Model config BartConfig {
  "_name_or_path": "gogamza/kobart-base-v1",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "

통과2
통과2


In [50]:
from tabulate import tabulate

In [51]:
print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,
            summaries_before_tuning,
        ),
        headers=["Id", "Summary after", "Summary before"],
    )
)
print("\nTarget summaries:\n")
print(
    tabulate(list(enumerate(test_samples["Summary"])), headers=["Id", "Target summary"])
)
print("\nSource documents:\n")
print(tabulate(list(enumerate(test_samples["Text"])), headers=["Id", "Document"]))

  Id  Summary after                                                                                                  Summary before
----  -------------------------------------------------------------------------------------------------------------  -------------------------------------------------------------------------------------------------------------
   0  - 아침 우리 임시공관의 발코니 위에 태극기를 게양하다. 성스러운                                                 - 아침 우리 임시공관의 발코니 위에 태극기를 게양하다. 성스러운
   1  그런 점에서 볼 때 박영민씨 경우는 이례적인 케이스로 꼽힌다. 프로게                                             그런 점에서 볼 때 박영민씨 경우는 이례적인 케이스로 꼽힌다. 프로게
   2  지난 지난 세계경제포럼 은 140개국을 상대로 한 금융시장 성숙도 조사에서 한국이 87                               지난 지난 세계경제포럼 은 140개국을 상대로 한 금융시장 성숙도 조사에서 한국이 87
   3  이처럼 이처럼 기금운용계획 변경범위를 축소한 것은 기금변경에 대한 국회의 심의대                                이처럼 이처럼 기금운용계획 변경범위를 축소한 것은 기금변경에 대한 국회의 심의대
   4  보이어 기장은 물체들의 실체를 파악하기 위해 더 가까이 가고 싶었지만 승객                                       보이어 기장은 물체들의 실체를 파악하기 위해 더 가까이 가고 싶었지만

In [44]:
tokenizer = RobertaTokenizerFast.from_pretrained("gogamza/kobart-base-v1")
model = EncoderDecoderModel.from_pretrained('result/checkpoint-3500/')
model.to("cuda")
batch_size = 32

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch
results = val_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["Text"])
pred_str = results["pred"]
label_str = results["Summary"]

loading file vocab.json from cache at /aiffel/.cache/huggingface/hub/models--gogamza--kobart-base-v1/snapshots/d7e64abd841bc1fa5d2939d14161124c51f29e8b/vocab.json
loading file merges.txt from cache at /aiffel/.cache/huggingface/hub/models--gogamza--kobart-base-v1/snapshots/d7e64abd841bc1fa5d2939d14161124c51f29e8b/merges.txt
loading file tokenizer.json from cache at /aiffel/.cache/huggingface/hub/models--gogamza--kobart-base-v1/snapshots/d7e64abd841bc1fa5d2939d14161124c51f29e8b/tokenizer.json
loading file added_tokens.json from cache at /aiffel/.cache/huggingface/hub/models--gogamza--kobart-base-v1/snapshots/d7e64abd841bc1fa5d2939d14161124c51f29e8b/added_tokens.json
loading file special_tokens_map.json from cache at /aiffel/.cache/huggingface/hub/models--gogamza--kobart-base-v1/snapshots/d7e64abd841bc1fa5d2939d14161124c51f29e8b/special_tokens_map.json
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /aiffel/.cache/huggingface/hub

OSError: Can't load the configuration of 'result/checkpoint-3500/'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'result/checkpoint-3500/' is the correct path to a directory containing a config.json file

In [ ]:
for num in range(500,800,10):
    print('predicted sentence : ',pred_str[num])
    print('real sentence : ', label_str[num])
    print('-'*30)

In [24]:
tokenizer = RobertaTokenizerFast.from_pretrained("klue/roberta-base")

tokenizer.bos_token = tokenizer.cls_token
print(tokenizer.bos_token)
tokenizer.eos_token = tokenizer.sep_token
print(tokenizer.eos_token)
#parameter setting
batch_size=32  #
encoder_max_length=256
decoder_max_length=64

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


[CLS]
[SEP]


In [25]:
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["Summary"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()

    # because RoBERTa automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
    # We have to make sure that the PAD token is ignored
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch

In [26]:
#processing training data
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"])
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],)

#processing validation data
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"])
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],)

  0%|          | 0/459 [00:00<?, ?ba/s]

  0%|          | 0/48 [00:00<?, ?ba/s]

In [27]:
type(train_data[0])

dict

In [28]:
train_data[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'decoder_attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
      

In [29]:
# 인코더 모델 불러오기

from transformers import EncoderDecoderModel

roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained("klue/roberta-base", "klue/roberta-base", tie_encoder_decoder=True)

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for

The following encoder weights were not tied to the decoder ['roberta/pooler']


In [30]:
# set special tokens
from transformers import EncoderDecoderConfig
roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id                                             
roberta_shared.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
roberta_shared.config.max_length = 128
roberta_shared.config.early_stopping = True
roberta_shared.config.no_repeat_ngram_size = 3
roberta_shared.config.length_penalty = 2.0
roberta_shared.config.num_beams = 4
roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size

In [31]:
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    num_train_epochs = 10,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    do_train=True,
    do_eval=True,
    logging_steps=500, 
    save_steps=16, 
    eval_steps=500, 
    warmup_steps=500, 
    overwrite_output_dir=True,
    save_total_limit=1,
    fp16=True,)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=roberta_shared,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,)
trainer.train()

Using cuda_amp half precision backend
The `config.pad_token_id` is `None`. Using `config.eos_token_id` = 2 for padding..
/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14687
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4590
  Number of trainable parameters = 139609088
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the 

Step,Training Loss
500,0.251200
1000,0.002800


Saving model checkpoint to ./checkpoint-16
Configuration saved in ./checkpoint-16/config.json
Model weights saved in ./checkpoint-16/pytorch_model.bin
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to ./checkpoint-32
Configuration saved in ./checkpoint-32/config.json
Model weights saved in ./checkpoint-32/pytorch_model.bin
Deleting older checkpoint [checkpoint-16] due to args.save_total_limit
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_

Saving model checkpoint to ./checkpoint-192
Configuration saved in ./checkpoint-192/config.json
Model weights saved in ./checkpoint-192/pytorch_model.bin
Deleting older checkpoint [checkpoint-176] due to args.save_total_limit
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to ./checkpoint-208
Configuration saved in ./checkpoint-208/config.json
Model weights saved in ./checkpoint-208/pytorch_model.bin
Deleting older checkpoint [checkpoint-192] due to args.save_total_limit
/opt/

Saving model checkpoint to ./checkpoint-368
Configuration saved in ./checkpoint-368/config.json
Model weights saved in ./checkpoint-368/pytorch_model.bin
Deleting older checkpoint [checkpoint-352] due to args.save_total_limit
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to ./checkpoint-384
Configuration saved in ./checkpoint-384/config.json
Model weights saved in ./checkpoint-384/pytorch_model.bin
Deleting older checkpoint [checkpoint-368] due to args.save_total_limit
/opt/

Saving model checkpoint to ./checkpoint-544
Configuration saved in ./checkpoint-544/config.json
Model weights saved in ./checkpoint-544/pytorch_model.bin
Deleting older checkpoint [checkpoint-528] due to args.save_total_limit
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to ./checkpoint-560
Configuration saved in ./checkpoint-560/config.json
Model weights saved in ./checkpoint-560/pytorch_model.bin
Deleting older checkpoint [checkpoint-544] due to args.save_total_limit
/opt/

Saving model checkpoint to ./checkpoint-720
Configuration saved in ./checkpoint-720/config.json
Model weights saved in ./checkpoint-720/pytorch_model.bin
Deleting older checkpoint [checkpoint-704] due to args.save_total_limit
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to ./checkpoint-736
Configuration saved in ./checkpoint-736/config.json
Model weights saved in ./checkpoint-736/pytorch_model.bin
Deleting older checkpoint [checkpoint-720] due to args.save_total_limit
/opt/

Saving model checkpoint to ./checkpoint-896
Configuration saved in ./checkpoint-896/config.json
Model weights saved in ./checkpoint-896/pytorch_model.bin
Deleting older checkpoint [checkpoint-880] due to args.save_total_limit
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to ./checkpoint-912
Configuration saved in ./checkpoint-912/config.json
Model weights saved in ./checkpoint-912/pytorch_model.bin
Deleting older checkpoint [checkpoint-896] due to args.save_total_limit
/opt/

Saving model checkpoint to ./checkpoint-1072
Configuration saved in ./checkpoint-1072/config.json
Model weights saved in ./checkpoint-1072/pytorch_model.bin
Deleting older checkpoint [checkpoint-1056] due to args.save_total_limit
/opt/conda/lib/python3.9/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
Saving model checkpoint to ./checkpoint-1088
Configuration saved in ./checkpoint-1088/config.json
Model weights saved in ./checkpoint-1088/pytorch_model.bin
Deleting older checkpoint [checkpoint-1072] due to args.save_total_lim

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained("gogamza/kobart-base-v1")
model = EncoderDecoderModel.from_pretrained('/aiffel/aiffel/aiffelthon/checkpoint-2752/')
model.to("cuda")
batch_size = 32

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch
results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["Text"])
pred_str = results["pred"]
label_str = results["Summary"]

In [ ]:
for num in range(500,800,10):
    print('predicted sentence : ',pred_str[num])
    print('real sentence : ', label_str[num])
    print('-'*30)